In [ ]:
# run once
!pip install pyreadstat lightgbm scikit-learn --quiet
print("Installed packages.")


Installed packages.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pyreadstat
import os

# ===== EDIT THIS path to your XPT in Drive =====
file_path = "/content/drive/MyDrive/LLCP2020.XPT"

assert os.path.exists(file_path), f"File not found: {file_path}"
print("Loading XPT (this may take a moment)...")
df, meta = pyreadstat.read_xport(file_path)
print("Raw XPT loaded. Shape:", df.shape)
print("Columns preview:", list(df.columns)[:60])


Loading XPT (this may take a moment)...
Raw XPT loaded. Shape: (401958, 279)
Columns preview: ['_STATE', 'FMONTH', 'IDATE', 'IMONTH', 'IDAY', 'IYEAR', 'DISPCODE', 'SEQNO', '_PSU', 'CTELENM1', 'PVTRESD1', 'COLGHOUS', 'STATERE1', 'CELPHONE', 'LADULT1', 'COLGSEX', 'NUMADULT', 'LANDSEX', 'NUMMEN', 'NUMWOMEN', 'RESPSLCT', 'SAFETIME', 'CTELNUM1', 'CELLFON5', 'CADULT1', 'CELLSEX', 'PVTRESD3', 'CCLGHOUS', 'CSTATE1', 'LANDLINE', 'HHADULT', 'SEXVAR', 'GENHLTH', 'PHYSHLTH', 'MENTHLTH', 'POORHLTH', 'HLTHPLN1', 'PERSDOC2', 'MEDCOST', 'CHECKUP1', 'EXERANY2', 'SLEPTIM1', 'CVDINFR4', 'CVDCRHD4', 'CVDSTRK3', 'ASTHMA3', 'ASTHNOW', 'CHCSCNCR', 'CHCOCNCR', 'CHCCOPD2', 'HAVARTH4', 'ADDEPEV3', 'CHCKDNY2', 'DIABETE4', 'DIABAGE3', 'LASTDEN4', 'RMVTETH4', 'MARITAL', 'EDUCA', 'RENTHOM1']


In [ ]:
# The 47 features list (adjust if your file uses slightly different names)
selected_cols = [
    "AGEG5YR","SEXVAR","BMI5","SMOKE100","SMOKDAY2","STOPSMK2","ALCDAY5","AVEDRNK2","DRNK3GE5","DIABETE4",
    "TOLDHI2","CVDCRHD4","CVDSTRK3","CHCCOPD1","HAVARTH3","CHCKIDNY","ADDEPEV3","MENTHLTH","PHYSHLTH","HLTHPLN1",
    "PERSDOC2","MEDCOST","CHECKUP1","EXERANY2","SLEPTIM1","FLUSHOT7","PNEUVAC4","HIVTST7","GENHLTH","POORHLTH",
    "QLACTLM2","USEEQUIP","DEAF","BLIND","DECIDE","DIFFWALK","DIFFDRES","DIFFALON","EMPLOY1","EDUCA",
    "INCOME2","MARITAL","RENTHOM1","VETERAN3","CHILDREN","HCVU651"
]

target_col = "CVDINFR4"  # typical heart-attack field in BRFSS; we'll rename to 'HeartAttack'

# Check presence
present = [c for c in selected_cols if c in df.columns]
missing = [c for c in selected_cols if c not in df.columns]
print(f"{len(present)}/{len(selected_cols)} selected features found. Missing: {missing}")

# Also check target column variations and rename to 'HeartAttack'
if target_col not in df.columns:
    # try common variants
    possible_targets = ["CVDINFR4","CVDINFR","CVDINFR_4","CVDINFR4_X","CVDSTRK3","CVDINFR_4"]
    found = None
    for t in possible_targets:
        if t in df.columns:
            found = t; break
    if found:
        print("Using alternative target column:", found)
        target_col = found
    else:
        raise AssertionError("Target column not found. Please locate heart-attack column in your XPT and set target_col.")

# Build subset with only available selected cols (we keep only those that exist)
use_cols = [c for c in selected_cols if c in df.columns]
df_sub = df[use_cols + [target_col]].copy()
df_sub = df_sub.rename(columns={target_col: "HeartAttack"})
print("Subset shape:", df_sub.shape)
print("Subset columns:", df_sub.columns.tolist()[:60])


36/46 selected features found. Missing: ['AGEG5YR', 'BMI5', 'AVEDRNK2', 'TOLDHI2', 'CHCCOPD1', 'HAVARTH3', 'CHCKIDNY', 'QLACTLM2', 'USEEQUIP', 'HCVU651']
Subset shape: (401958, 37)
Subset columns: ['SEXVAR', 'SMOKE100', 'SMOKDAY2', 'STOPSMK2', 'ALCDAY5', 'DRNK3GE5', 'DIABETE4', 'CVDCRHD4', 'CVDSTRK3', 'ADDEPEV3', 'MENTHLTH', 'PHYSHLTH', 'HLTHPLN1', 'PERSDOC2', 'MEDCOST', 'CHECKUP1', 'EXERANY2', 'SLEPTIM1', 'FLUSHOT7', 'PNEUVAC4', 'HIVTST7', 'GENHLTH', 'POORHLTH', 'DEAF', 'BLIND', 'DECIDE', 'DIFFWALK', 'DIFFDRES', 'DIFFALON', 'EMPLOY1', 'EDUCA', 'INCOME2', 'MARITAL', 'RENTHOM1', 'VETERAN3', 'CHILDREN', 'HeartAttack']


In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Replace common BRFSS missing codes with NaN
df_sub = df_sub.replace({7: np.nan, 9: np.nan, 77: np.nan, 99: np.nan, 997: np.nan, 999: np.nan})

# Drop rows with NaN (paper did similar); keep same behavior
df_sub = df_sub.dropna().reset_index(drop=True)
print("After dropna shape:", df_sub.shape)

# Map target to {0,1} — BRFSS frequently uses 1=Yes, 2=No
raw_vals = np.unique(df_sub['HeartAttack'])
print("Raw HeartAttack values (sample):", raw_vals[:10])
y_raw = df_sub['HeartAttack'].to_numpy()
y = np.where(y_raw == 1, 1, 0)
print("Mapped target counts (0,1):", np.bincount(y))

# Final feature matrix
X_df = df_sub.drop(columns=['HeartAttack']).astype(np.float32)
feature_names = list(X_df.columns)
print("Using features (n):", len(feature_names))

# Fit MinMax scaler (same as paper)
scaler = MinMaxScaler()
X = scaler.fit_transform(X_df.values.astype(np.float32))
print("Final X shape:", X.shape)


After dropna shape: (5082, 37)
Raw HeartAttack values (sample): [1. 2.]
Mapped target counts (0,1): [4789  293]
Using features (n): 36
Final X shape: (5082, 36)


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import balanced_accuracy_score, recall_score, roc_auc_score, roc_curve
import numpy as np

def youdens_j_threshold(y_true, y_prob):
    fpr, tpr, thr = roc_curve(y_true, y_prob)
    j = tpr - fpr
    return thr[np.argmax(j)]

def fit_evaluate(model, X_arr, y_arr, n_splits=5, random_state=42):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    bal_accs, macro_recalls, recall1s, aucs, thrs = [], [], [], [], []
    for tr, te in skf.split(X_arr, y_arr):
        Xtr, Xte = X_arr[tr], X_arr[te]
        ytr, yte = y_arr[tr], y_arr[te]
        model.fit(Xtr, ytr)
        p = model.predict_proba(Xte)[:,1]
        thr = youdens_j_threshold(yte, p)
        thrs.append(thr)
        yhat = (p >= thr).astype(int)
        bal_accs.append(balanced_accuracy_score(yte, yhat))
        macro_recalls.append(recall_score(yte, yhat, average="macro", zero_division=0))
        recall1s.append(recall_score(yte, yhat, pos_label=1))
        aucs.append(roc_auc_score(yte, p))
    return {
        "Balanced Accuracy": float(np.mean(bal_accs)),
        "Macro Recall": float(np.mean(macro_recalls)),
        "Recall Class 1": float(np.mean(recall1s)),
        "AUC": float(np.mean(aucs)),
        "Youden Threshold": float(np.mean(thrs))
    }

print("Evaluation harness ready.")


Evaluation harness ready.


In [ ]:
import lightgbm as lgb

lgbm_base = lgb.LGBMClassifier(
    n_estimators=600,
    learning_rate=0.05,
    max_depth=-1,
    subsample=0.8,
    colsample_bytree=0.8,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

print("Running LightGBM baseline (5-fold CV)...")
baseline_res = fit_evaluate(lgbm_base, X, y)
for k,v in baseline_res.items():
    print(f"{k}: {v:.4f}")


Running LightGBM baseline (5-fold CV)...
[LightGBM] [Info] Number of positive: 234, number of negative: 3831
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001166 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 239
[LightGBM] [Info] Number of data points in the train set: 4065, number of used features: 35
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 234, number of negative: 3831
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001069 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 239
[LightGBM] [Info] Number of data points in the train set: 4065, number of used features: 35
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 235, number of negative: 3831
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000965 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 240
[LightGBM] [Info] Number of data points in the train set: 4066, number of used features: 35
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 235, number of negative: 3831
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001007 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 236
[LightGBM] [Info] Number of data points in the train set: 4066, number of used features: 35
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 234, number of negative: 3832
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001057 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 239
[LightGBM] [Info] Number of data points in the train set: 4066, number of used features: 35
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
Balanced Accuracy: 0.7442
Macro Recall: 0.7442
Recall Class 1: 0.7202
AUC: 0.8034
Youden Threshold: 0.0115


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
from sklearn.feature_selection import mutual_info_classif
TOP_K = 20

print("Computing mutual information to rank features...")
mi = mutual_info_classif(X, y, discrete_features=False, random_state=42)
import pandas as pd
mi_series = pd.Series(mi, index=feature_names).sort_values(ascending=False)
top_features = mi_series.head(TOP_K).index.tolist()
print("Top features (by MI):", top_features)


Computing mutual information to rank features...
Top features (by MI): ['CVDCRHD4', 'CVDSTRK3', 'PHYSHLTH', 'EMPLOY1', 'GENHLTH', 'DIABETE4', 'DEAF', 'DIFFWALK', 'PNEUVAC4', 'DRNK3GE5', 'HIVTST7', 'BLIND', 'POORHLTH', 'HLTHPLN1', 'CHECKUP1', 'PERSDOC2', 'EDUCA', 'MARITAL', 'INCOME2', 'ADDEPEV3']


In [ ]:
from itertools import combinations
cand_pairs = list(combinations(top_features, 2))
print("Candidate pairs count:", len(cand_pairs))
# quick sanity sample
print("Example candidate pairs:", cand_pairs[:10])


Candidate pairs count: 190
Example candidate pairs: [('CVDCRHD4', 'CVDSTRK3'), ('CVDCRHD4', 'PHYSHLTH'), ('CVDCRHD4', 'EMPLOY1'), ('CVDCRHD4', 'GENHLTH'), ('CVDCRHD4', 'DIABETE4'), ('CVDCRHD4', 'DEAF'), ('CVDCRHD4', 'DIFFWALK'), ('CVDCRHD4', 'PNEUVAC4'), ('CVDCRHD4', 'DRNK3GE5'), ('CVDCRHD4', 'HIVTST7')]


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import time, copy

def evaluate_with_pairs(pair_list, X_base, y_arr, n_splits=3):
    X_aug = X_base.copy()
    for (a,b) in pair_list:
        a_idx = feature_names.index(a)
        b_idx = feature_names.index(b)
        prod_col = (X_base[:, a_idx] * X_base[:, b_idx]).reshape(-1,1)
        X_aug = np.hstack([X_aug, prod_col])
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=123)
    aucs = []
    clf = lgb.LGBMClassifier(n_estimators=200, learning_rate=0.05, class_weight='balanced', random_state=42, n_jobs=-1)
    for tr, te in skf.split(X_aug, y_arr):
        clf.fit(X_aug[tr], y_arr[tr])
        p = clf.predict_proba(X_aug[te])[:,1]
        aucs.append(roc_auc_score(y_arr[te], p))
    return np.mean(aucs)

MAX_ADD = 10   # maximum number of interaction features to add
selected_pairs = []
remaining = cand_pairs.copy()
best_score = evaluate_with_pairs(selected_pairs, X, y)  # baseline proxy score
print("Initial proxy AUC (no interactions):", best_score)

for add_i in range(MAX_ADD):
    print(f"\nIteration {add_i+1}/{MAX_ADD}. Current selected pairs:", selected_pairs)
    best_candidate = None
    best_candidate_score = best_score
    start_it = time.time()
    for cand in remaining:
        trial_pairs = selected_pairs + [cand]
        s = evaluate_with_pairs(trial_pairs, X, y)
        # progress print for long runs (optional)
        # print(f"Tested {cand} -> proxy AUC {s:.5f}")
        if s > best_candidate_score + 1e-6:  # slight improvement threshold
            best_candidate_score = s
            best_candidate = cand
    end_it = time.time()
    if best_candidate is None:
        print("No further improvement found. Stopping selection.")
        break
    print(f"Selected pair {best_candidate} with proxy AUC {best_candidate_score:.5f} (improved from {best_score:.5f}). Time: {end_it-start_it:.1f}s")
    selected_pairs.append(best_candidate)
    remaining.remove(best_candidate)
    best_score = best_candidate_score

print("\nFinal selected pairs:", selected_pairs)


[LightGBM] [Info] Number of positive: 196, number of negative: 3192
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000822 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 233
[LightGBM] [Info] Number of data points in the train set: 3388, number of used features: 35
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 195, number of negative: 3193
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000819 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 234
[LightGBM] [Info] Number of data points in the train set: 3388, number of used features: 35
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 195, number of negative: 3193
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000820 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 237
[LightGBM] [Info] Number of data points in the train set: 3388, number of used features: 35
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
Initial proxy AUC (no interactions): 0.8087689112278692

Iteration 1/10. Current selected pairs: []
[LightGBM] [Info] Number of positive: 196, number of negative: 3192
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000863 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 235
[LightGBM] [Info] Numb

...

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 195, number of negative: 3193
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.178431 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 322
[LightGBM] [Info] Number of data points in the train set: 3388, number of used features: 38
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 195, number of negative: 3193
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000835 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 323
[LightGBM] [Info] Number of data points in the train set: 3388, number of used features: 38
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
No further improvement found. Stopping selection.

Final selected pairs: [('DEAF', 'DRNK3GE5'), ('EMPLOY1', 'DRNK3GE5')]


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
# Build final augmented feature matrix
X_aug = X.copy()
aug_feature_names = feature_names.copy()
for (a,b) in selected_pairs:
    a_idx = feature_names.index(a)
    b_idx = feature_names.index(b)
    prod_col = (X[:, a_idx] * X[:, b_idx]).reshape(-1,1)
    X_aug = np.hstack([X_aug, prod_col])
    aug_feature_names.append(f"{a}__x__{b}")

print("Augmented X shape:", X_aug.shape)
print("Augmented feature names tail:", aug_feature_names[-len(selected_pairs):])


Augmented X shape: (5082, 38)
Augmented feature names tail: ['DEAF__x__DRNK3GE5', 'EMPLOY1__x__DRNK3GE5']


In [ ]:
lgbm_aug = lgb.LGBMClassifier(
    n_estimators=600,
    learning_rate=0.05,
    max_depth=-1,
    subsample=0.8,
    colsample_bytree=0.8,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

print("Running final 5-fold CV on augmented features...")
aug_res = fit_evaluate(lgbm_aug, X_aug, y)
print("\n=== Final results with selected interaction features ===")
for k,v in aug_res.items():
    print(f"{k}: {v:.4f}")


Running final 5-fold CV on augmented features...
[LightGBM] [Info] Number of positive: 234, number of negative: 3831
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.080059 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 320
[LightGBM] [Info] Number of data points in the train set: 4065, number of used features: 37
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 234, number of negative: 3831
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001164 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 318
[LightGBM] [Info] Number of data points in the train set: 4065, number of used features: 37
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 235, number of negative: 3831
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000976 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 321
[LightGBM] [Info] Number of data points in the train set: 4066, number of used features: 37
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 235, number of negative: 3831
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001183 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 316
[LightGBM] [Info] Number of data points in the train set: 4066, number of used features: 37
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 234, number of negative: 3832
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001250 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 322
[LightGBM] [Info] Number of data points in the train set: 4066, number of used features: 37
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000

=== Final results with selected interaction features ===
Balanced Accuracy: 0.7496
Macro Recall: 0.7496
Recall Class 1: 0.7469
AUC: 0.8143
Youden Threshold: 0.0081


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [23]:
import pandas as pd
from IPython.display import display

# Combine baseline vs augmented results into a table
results_table = pd.DataFrame([baseline_res, aug_res], index=["Baseline LGBM", "LGBM + Interactions"])

print("📊 Final Results Comparison")
display(
    results_table.style
        .set_table_styles(
            [{'selector': 'th', 'props': [('font-size', '14px'), ('text-align', 'center')]},
             {'selector': 'td', 'props': [('font-size', '13px'), ('text-align', 'center')]}]
        )
        .format(precision=4)
        .set_caption("Model Performance (5-Fold CV)")
)


📊 Final Results Comparison


,Balanced Accuracy,Macro Recall,Recall Class 1,AUC,Youden Threshold
Baseline LGBM,0.7442,0.7442,0.7202,0.8034,0.0115
LGBM + Interactions,0.7496,0.7496,0.7469,0.8143,0.0081
